In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [2]:
from transformers import BertModel, BertConfig
from transformers.models.bert.modeling_bert import *

In [3]:
from new.networks import *
from new.utils import *
from new.models import *

In [4]:
from options.options import Options
import os
import torch
from tqdm.auto import tqdm

In [5]:
from build_dataset_model import build_suncg_dsets
from data.suncg_dataset import SuncgDataset

In [6]:
args = Options().parse()
if (args.output_dir is not None) and (not os.path.isdir(args.output_dir)):
    os.mkdir(args.output_dir)
if (args.test_dir is not None) and (not os.path.isdir(args.test_dir)):
    os.mkdir(args.test_dir)

| options
dataset: suncg
suncg_train_dir: metadata/data_rot_train.json
suncg_val_dir: metadata/data_rot_val.json
suncg_data_dir: /home/yizhou/Research/SUNCG/suncg_data
loader_num_workers: 8
embedding_dim: 64
gconv_mode: feedforward
gconv_dim: 128
gconv_hidden_dim: 512
gconv_num_layers: 5
mlp_normalization: batch
vec_noise_dim: 0
layout_noise_dim: 32
batch_size: 16
num_iterations: 60000
eval_mode_after: -1
learning_rate: 0.0001
print_every: 100
checkpoint_every: 1000
snapshot_every: 10000
output_dir: ./checkpoints
checkpoint_name: latest_checkpoint
timing: False
multigpu: False
restore_from_checkpoint: False
checkpoint_start_from: None
test_dir: ./layouts_out
gpu_id: 0
KL_loss_weight: 0.1
use_AE: False
decoder_cat: True
train_3d: True
KL_linear_decay: False
use_attr_30: True
manual_seed: 42
batch_gen: False
measure_acc_l1_std: False
heat_map: False
draw_2d: False
draw_3d: False
fine_tune: False
gan_shade: False
blender_path: /home/yizhou/blender-2.92.0-linux64/blender



In [7]:
dset_kwargs = {
        'data_dir': args.suncg_train_dir,
        'train_3d': args.train_3d,
        'use_attr_30': args.use_attr_30,
    }

In [8]:
train_dset = SuncgDataset(**dset_kwargs)

Starting to read the json file for SUNCG
loading relation score matrix from:  new/relation_graph_v1.p


In [9]:
train_dset.vocab

{'object_idx_to_name': ['__room__',
  'curtain',
  'shower_curtain',
  'dresser',
  'counter',
  'bookshelf',
  'picture',
  'mirror',
  'floor_mat',
  'chair',
  'sink',
  'desk',
  'table',
  'lamp',
  'door',
  'clothes',
  'person',
  'toilet',
  'cabinet',
  'floor',
  'window',
  'blinds',
  'wall',
  'pillow',
  'whiteboard',
  'bathtub',
  'television',
  'night_stand',
  'sofa',
  'refridgerator',
  'bed',
  'shelves'],
 'object_name_to_idx': {'__room__': 0,
  'curtain': 1,
  'shower_curtain': 2,
  'dresser': 3,
  'counter': 4,
  'bookshelf': 5,
  'picture': 6,
  'mirror': 7,
  'floor_mat': 8,
  'chair': 9,
  'sink': 10,
  'desk': 11,
  'table': 12,
  'lamp': 13,
  'door': 14,
  'clothes': 15,
  'person': 16,
  'toilet': 17,
  'cabinet': 18,
  'floor': 19,
  'window': 20,
  'blinds': 21,
  'wall': 22,
  'pillow': 23,
  'whiteboard': 24,
  'bathtub': 25,
  'television': 26,
  'night_stand': 27,
  'sofa': 28,
  'refridgerator': 29,
  'bed': 30,
  'shelves': 31},
 'pred_idx_to_na

In [10]:
train_dset[0]

(4377,
 tensor([ 9, 18, 20, 30, 22,  3, 14, 26, 27, 26,  0]),
 tensor([[ 1.2971e-01,  1.8182e-02,  6.6184e-01,  4.3746e-01,  3.2920e-01,
           8.7699e-01],
         [-1.8332e-02,  6.7337e-01,  5.6516e-01,  1.9248e-01,  7.8162e-01,
           5.9611e-01],
         [ 7.1195e-01, -5.4443e-16,  5.0570e-01,  1.2310e+00,  5.0909e-01,
           5.5158e-01],
         [ 2.2797e-01,  1.8182e-02, -1.0230e-01,  6.3582e-01,  2.9879e-01,
           4.4557e-01],
         [ 8.4345e-01,  1.8182e-02,  4.0563e-02,  9.5468e-01,  1.0000e+00,
           1.1921e-01],
         [ 6.6919e-01,  1.8182e-02,  4.8725e-01,  1.1141e+00,  3.4545e-01,
           5.6704e-01],
         [ 6.6050e-01,  7.2728e-03,  7.8242e-01,  9.3001e-01,  7.7818e-01,
           1.0000e+00],
         [ 5.3706e-02,  2.1091e-01,  3.5881e-01,  1.7235e-01,  2.7273e-01,
           4.1805e-01],
         [ 4.6290e-02,  1.8182e-02,  3.2368e-01,  1.9460e-01,  2.1091e-01,
           4.4950e-01],
         [ 5.9412e-02,  6.3743e-01,  9.3840e-01

In [11]:
args.loader_num_workers

8

In [12]:
loader_kwargs = {
        'batch_size': args.batch_size,
        'num_workers': 1, #args.loader_num_workers,
        'shuffle': True,
        'collate_fn': new_collate_fn,
    }

In [13]:
train_loader = DataLoader(train_dset, **loader_kwargs)

In [14]:
fetgg = FromEncoderToGraphGenerator()

In [15]:
use_cuda = True

In [16]:
if use_cuda:
    fetgg = fetgg.cuda()

In [17]:
fetgg_optim = torch.optim.Adam(fetgg.parameters(),lr = 1e-5)

In [18]:
import numpy as np

In [21]:
torch.device("cuda:0")

device(type='cuda', index=0)

In [ ]:
train_loss = []
for c, batch in tqdm(enumerate(train_loader)):
    objs, boxes, angles, attention_mask = batch[0]
    if use_cuda:
        objs = objs.to(torch.device("cuda:0"))
        boxes = boxes.to(torch.device("cuda:0"))
        angles = angles.to(torch.device("cuda:0"))
        attention_mask = attention_mask.to(torch.device("cuda:0"))
    
    logits, loss = output = fetgg(objs, boxes, angles, attention_mask)
    
    fetgg_optim.zero_grad()
    loss.backward()
    fetgg_optim.step()
    
    train_loss.append(loss.item())
    if c % 100 == 0:
        print(c // 100, np.mean(train_loss))
        train_loss.clear()

0it [00:00, ?it/s]

0 3.5646960735321045
1 2.8788965666294097
2 1.768669775724411
3 1.3444814032316208
4 1.2780265271663667
5 1.265349373817444
6 1.2641550076007844
7 1.2743601176142692
8 1.183803898692131
9 1.1791972780227662
10 1.2004164871573448
11 1.1448963797092437
12 1.226352010667324
13 1.2384966117143632
14 1.193520284295082
15 1.1459601354598998
16 1.1931898367404938
17 1.1592135655879974
18 1.190635159611702
